### 6.2.1 contextual bandit 问题
#### 一. web推荐系统存在的问题
1. 基于web的场景, 物品经常会变, 物品的流行度也会变化. 大量的新用户也没有历史消费信息.  
2. 传统的协同过滤矩阵分解算法, 需要求解出物品和用户的隐因子向量, 这很耗费时间, 会降低用户的短期体验.  
 因此, 我们要平衡2件事 : 最大化用户的长期满意度和学习到最佳的用户物品隐因子向量
 
#### 二. contextual bandit 问题抽象
1. `个性化新闻标题推荐`问题可抽象为带上下文的多臂赌博机问题  
 `多臂赌博机`问题是在t次试验中进行的 : 
   1. 设当前用户$u_t$和臂集合${ \Lambda   }_{ t }$, 对于每一个臂$\alpha \in { \Lambda  }_{ t }$, 向量${ x }_{ t,a }$是综合了用户$u_t$和臂$\alpha$的表示, 这种向量表示称为context  
   2. 基于先前试验的结果, 使用$r_{t,{\alpha}_t}$表示用户$u_t$在${\alpha}_t$上的奖励 
   3. 在下一次实验中, 根据3个参数$\left( { x }_{ t,{ \alpha  }_{ t } },{ \alpha  }_{ t },{ r }_{ t,{ \alpha  }_{ t } } \right) $更新臂选择策略  
    值得注意的是, 没有被选择的臂$\left( \alpha \neq { \alpha  }_{ t } \right) $就没有反馈
2. 最优化问题  
t次实验的收益$A$, 定义为$A=\sum _{ t=1 }^{ T }{ { r }_{ t,{ \alpha  }_{ t } } } $.   
同理, 我们的最佳t次实验收益为$E\left[ \sum _{ t=1 }^{ T }{ { r }_{ t,{ \alpha  }_{ t }^{ * } } }  \right] $, ${ \alpha  }_{ t }^{ * }$为第t次实验的最佳臂选择. 即最小化损失$${ R }_{ A }\left( T \right) =E\left[ \sum _{ t=1 }^{ T }{ { r }_{ t,{ \alpha  }_{ t }^{ * } } }  \right] -E\left[ \sum _{ t=1 }^{ T }{ { r }_{ t,{ \alpha  }_{ t } } }  \right] $$
3. 本文中, 我们聚焦于新闻标题推荐场景. 该场景可抽象为, 用户对新闻标题的点击绿预测问题(CTR).  
 结合bandit算法, 如果用户对推荐的标题点击了, 则回馈为1, 否则, 回馈为0
4. 对于推荐算法的EE问题(explore&exploit), 实质就是, 我们为什么要选择次最优的臂?  
 因为次最优的臂, 会让系统学习到更多的信息. 而我们判定出的最优臂,可能是因为信息获取不足造成的偏差结果. 选择次最优的臂, 可能会造成短期满意度下降, 但却会提高长期满意度. 例如, 不带上下文的bandit算法中, $\varepsilon -greedy$就是考虑了EE问题的算法

### 6.2.2 算法

#### 一. LinUCB算法总览
1. 我们将看到, 在计算收益时, 如果使用线性模型, 则置信区间可以高效算出

#### 二. LinUCB - 使用互斥线性模型
1. 假设有臂$\alpha$,$d$维context向量${ x }_{ t,\alpha  }$,未知参数${ \theta  }_{ \alpha  }^{ * }$, 则有期望收益$$E\left[ { r }_{ t,{ \alpha  } }|{ x }_{ t,a } \right] =\left( { { x }_{ t,\alpha  } } \right) ^{ T }{ \theta  }_{ \alpha  }^{ * }$$  
  之所以称该模型为互斥线性模型, 是因为不同臂之间的参数${ \theta  }_{ \alpha  }$并不共享  
2. 参数求解: 在第t次试验里  
  1. ${ D }_{ \alpha  }$ : $R^{m*d}$维矩阵. 每行作为一个输入臂$\alpha$的context, 即矩阵D的每一行是$x_{t,\alpha}$  
  2. $ b_{ \alpha  }$ : $ { R }^{ m }$向量, 是臂$\alpha$的反馈向量(用户点击/未点击). ${ c }_{ \alpha  }$的每一行由向量${ b }_{ \alpha  }$组成  
  3. 在训练数据$\left( { D }_{ \alpha  },{ c }_{ \alpha  } \right) $上, 使用岭回归估计参数:$${ loss }_{ all-item }\left( \theta  \right) ={ \left( D{ \theta  }-{ c }_{ \alpha  } \right)  }^{ 2 }+{ \left\| \theta  \right\|  }^{ 2 }\\ \therefore \frac { \partial loss }{ \partial \theta  } =2D\left( D{ \theta  }-{ c }_{ \alpha  } \right) +2\theta =0\\ \quad \quad \quad \quad \quad =\left( { D }^{ T }D+{ I }_{ d } \right) \theta =D{ c }_{ \alpha  }\\ \therefore \hat { { \theta  } } ={ \left( { D }^{ T }D+{ I }_{ d } \right)  }^{ -1 }D{ c }_{ \alpha  }$$
3. 置信区间  
 以上参数估计得出的$\hat { { \theta  } } $, 使得计算得到的奖励和最有奖励之间的差距, 在如下置信区间内:([推导过程](https://arxiv.org/pdf/1205.2606.pdf))
 <img src='img/linucb1.png' height='40%' width='40%'>
4. 臂的选择 
<img src='img/linucb2.png' height='35%' width='35%'>

#### 三. 混合线性模型的UCB
1. 在新闻推荐场景中, 一个用户可能有自己的口特点. 这个口味在各个臂的选择中都会起到作用, 因此, 需要在计算奖励时, 在每个臂的奖励中加上共享参数${ \beta  }$ ,即得到混合线性型的UCB的奖励计算公式 : $$E\left[ { r }_{ t,\alpha  }|{ x }_{ t,\alpha  } \right] ={ z }_{ t,\alpha  }^{ T }{ \beta  }^{ * }+{ x }_{ t,\alpha  }^{ T }{ \theta  }_{ \alpha  }^{ * }$$
2. 模型实现  
 算法包含五个参数: $\left( { A }_{ 0 },{ b }_{ 0 },{ A }_{ \alpha  },{ B }_{ \alpha  },{ b }_{ \alpha  } \right) $
<img src='img/hybirdlinucb.png' width='50%' height='50%'>